### Praktikum 2
Generator Teks dengan RNN

In [ ]:
# Import library
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


### Olah Teks
Vectorize Teks

In [ ]:
# contoh teks yang akan dipecah menjadi karakter
example_texts = ['abcdefg', 'xyz']

# pecah setiap teks menjadi karakter menggunakan encoding UTF-8
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

In [ ]:
# buat layer StringLookup untuk mengonversi karakter ke ID
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
# Menggunakan layer StringLookup untuk mengonversi karakter menjadi ID
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
# buat layer StringLookup untuk mengonversi ID kembali ke karakter
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars=chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# menggabungkan karakter-karakter menjadi teks menggunakan reduce_join
tf.strings.reduce_join(chars, axis=-1).numpy()

# fungsi untuk mengonversi ID menjadi teks
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Prediksi
Membuat Trianing Set dan Target

In [ ]:
# mengonversi seluruh teks menjadi ID menggunakan StringLookup
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset=tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
# Menampilkan 10 contoh teks dari dataset ID
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

# Panjang urutan yang diinginkan
seq_length = 100

F
i
r
s
t
 
C
i
t
i


In [ ]:
# Membuat urutan (sequences) dari dataset ID dengan panjang urutan + 1
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

# Menampilkan satu contoh urutan
for seq in sequences.take(1):
    print(chars_from_ids(seq))


tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
# menampilkan teks dari lima contoh urutan
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
# Fungsi untuk memisahkan teks input dan teks target dari suatu urutan
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
# Memisahkan teks input dan teks target dari urutan karakter "Tensorflow"
split_input_target(list("Tensorflow"))

# Membuat dataset dari urutan karakter dengan fungsi split_input_target
dataset = sequences.map(split_input_target)

# Menampilkan contoh input dan target dari dataset
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Batch Training

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Buat Model

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
# Membuat instance dari model kustom (MyModel)
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


Uji Model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Menghasilkan indeks sampel dengan menggunakan distribusi kategori acak
sampled_indices=tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices= tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

print("Input:\n",text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'hou king, and wilt be forced?\nI shame to hear thee speak. Ah, timorous wretch!\nThou hast undone thys'

Next Char Predictions:
 b'cClv\n;.nzr Xb-iDAkP3f!EuWs.GEZv\nSfn;YDD-gGCxUJSZKUu&-i$pa.gXsB?[UNK]bd$DNAM:WHoolooLX:P $VjHJZUbrQ-\niO[UNK]C'


Train Model

Tambahan optimizer dan fungsi loss

In [ ]:
# Membuat fungsi loss menggunakan Sparse Categorical Crossentropy
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Menghitung rata-rata loss menggunakan Sparse Categorical Crossentropy
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189146, shape=(), dtype=float32)


In [ ]:
# Menghitung nilai eksponensial dari rata-rata kerugian
tf.exp(example_batch_mean_loss).numpy()
# Mengompilasi model dengan optimizer Adam
model.compile(optimizer='adam', loss=loss)

Konfigurasi Checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Lakukan Proses Training

In [ ]:
# Jumlah epoch yang diinginkan
EPOCHS = 20

# Melatih model dengan dataset dan callback untuk menyimpan checkpoint
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 17s 56ms/step - loss: 2.7349
Epoch 2/20
172/172 [==============================] - 13s 56ms/step - loss: 2.0033
Epoch 3/20
172/172 [==============================] - 13s 60ms/step - loss: 1.7221
Epoch 4/20
172/172 [==============================] - 15s 60ms/step - loss: 1.5602
Epoch 5/20
172/172 [==============================] - 12s 60ms/step - loss: 1.4600
Epoch 6/20
172/172 [==============================] - 12s 60ms/step - loss: 1.3898
Epoch 7/20
172/172 [==============================] - 12s 60ms/step - loss: 1.3362
Epoch 8/20
172/172 [==============================] - 12s 60ms/step - loss: 1.2920
Epoch 9/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2517
Epoch 10/20
172/172 [==============================] - 13s 59ms/step - loss: 1.2123
Epoch 11/20
172/172 [==============================] - 16s 65ms/step - loss: 1.1729
Epoch 12/20
172/172 [==============================] - 15s 60ms/step - loss: 1.1328
E

Generate Teks

In [ ]:
# Definisi class OneStep
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Meletakkan -inf pada setiap indeks yang tidak diinginkan.
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,
            # Sesuaikan bentuk dengan kosakata
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Mengonversi string menjadi token ID.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Menjalankan model.
        # Bentuk predicted_logits adalah [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Hanya menggunakan prediksi terakhir.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Terapkan mask prediksi
        predicted_logits = predicted_logits + self.prediction_mask

        # Ambil sampel token ID keluaran.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Mengonversi dari token ID menjadi karakter.
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Mengembalikan karakter dan state model.
        return predicted_chars, states


In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
But had he died, and many a match.

MERCUTIO:
And, by God's!

COMINIUS:
Ho! no word.

AUFODYCUS:
I had a bady school-dangerous lie!
Camills fantagenet, and the bear most unnatural aid
And crows too talk alone: let this cliff;
Lead this safetial, and importuned men
As poist or women: every sorrow has the king's,
And give us not, too fair and very shot.

LUCIO:
Well, I must I had slain Lewis' to Calling means:
To be so ruled, to pluck all former lands,
I'll make the ears, and some care.

Gardenes: let
them by the house: where I should not be sleeps?

TRANIO:
Was ever follow'd this before you well?

LUCIO:
That's the heart that cannot do it.

ESCALUS:
Everyore, sir! none.

PETRUCHIO:
You have a bard air of me, my sword in the
disguision of us thus I buckle,
We'll much ever up to strike? it may enter
And over all discord lain with subs and will not
Come hither: sirs, content a tale, sir.

FLORIZEL:
My boy,
Done gift than feet to this new marmless in his errand;
My way is the younger

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)

tf.Tensor(
[b"ROMEO:\nThou shouldst have marriage now,\nWhom I, by repeal, my his own life and mine\nShall rest with traitor, provost; and myself\nBonest in earnestness to's throats of time,\nSome are by Romeo, to see thee all.\n\nBIONDELLO:\nI give you in you, sir; I'll not be so bad and a cold;\nAnd this before my good sister, but my sovereign\nknels a miserable sly; only to be Lond of Grounca,\nWhich severeivy to subscurns! I am too so.\n\nKING RICHARD III:\nKind conforn, my brother brings the triumph day.\n\nDUKE OF AUMERLE:\nFor my ware do London with that stones at will make his mother\nshould be talked for better at that counsel in the scene.\nHark ye, and be the better.\n\nMy needs you are we sword spirit to Lodd of George,\nAnd thou some queen's are the night is dege.\n\nSLY:\nI am could never welcome a. Behold, this troable to hear\nAs I am a most needful briteful body.\n\nVIRGILIA:\nSir, a little me, encounted, as it were Mistress;\nAnd Mercutio's dead likes, under laug-heat

Ekspor Model Generator

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Well, by my mistress, pardon, we'll not push death;
And thou didst kill the king and that I may not


# **Tugas**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan tf.GradientTape untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca eager execution guide.

Prosedurnya adalah "

1. Jalankan Model dan hitung loss dengan tf.GradientTape.

2. Hitung update dan terapkan pada model dengan optimizer

In [ ]:
# Mendefinisikan class CustomTraining yang merupakan turunan dari MyModel
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

Kode diatas menerapkan train_step method sesuai dengan  Keras' train_step conventions. Ini opsional, tetapi memungkinkan Anda mengubah perilaku langkah pelatihan dan tetap menggunakan keras Model.compile and Model.fit methods.

In [ ]:
# Membuat instance dari kelas CustomTraining
model = CustomTraining(

    # Menentukan ukuran kosakata (vocab_size) berdasarkan panjang kosakata yang dihasilkan oleh ids_from_chars.get_vocabulary()
    vocab_size=len(ids_from_chars.get_vocabulary()),

    # Menentukan dimensi embedding (embedding_dim) untuk lapisan embedding model
    embedding_dim=embedding_dim,

    # Menentukan jumlah unit dalam lapisan GRU (rnn_units) pada model
    rnn_units=rnn_units)

In [ ]:
# Mengompilasi model dengan pengoptimal Adam dan fungsi loss Sparse Categorical Crossentropy
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [ ]:
# Melatih model dengan dataset untuk satu epoch
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 60ms/step - loss: 2.7308


Atau jika ingin lebih mengetahui dalamnya, kita bisa membuat custom training loop sendiri:

In [ ]:
# Jumlah epoch yang diinginkan
EPOCHS = 10

# Inisialisasi mean untuk menghitung rata-rata loss
mean = tf.metrics.Mean()

# Melakukan iterasi melalui epoch
for epoch in range(EPOCHS):
    start = time.time()

    # Mengatur ulang state mean
    mean.reset_states()

    # Melakukan iterasi melalui batch-batch dalam dataset
    for (batch_n, (inp, target)) in enumerate(dataset):
        # Memperbarui state dan mendapatkan logs dari model.train_step
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        # Menampilkan informasi setiap 50 batch
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # Menyimpan (checkpoint) model setiap 5 epoch
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

# Menyimpan berat model setelah selesai pelatihan
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1438
Epoch 1 Batch 50 Loss 2.0615
Epoch 1 Batch 100 Loss 1.9403
Epoch 1 Batch 150 Loss 1.8454

Epoch 1 Loss: 1.9991
Time taken for 1 epoch 13.15 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8367
Epoch 2 Batch 50 Loss 1.7609
Epoch 2 Batch 100 Loss 1.6990
Epoch 2 Batch 150 Loss 1.7070

Epoch 2 Loss: 1.7285
Time taken for 1 epoch 11.70 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5993
Epoch 3 Batch 50 Loss 1.6000
Epoch 3 Batch 100 Loss 1.5387
Epoch 3 Batch 150 Loss 1.5154

Epoch 3 Loss: 1.5677
Time taken for 1 epoch 12.33 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4476
Epoch 4 Batch 50 Loss 1.4889
Epoch 4 Batch 100 Loss 1.4579
Epoch 4 Batch 150 Loss 1.3921

Epoch 4 Loss: 1.4661
Time taken for 1 epoch 12.53 sec
_____________________________________________________________________

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

**Perbandingan:**

- Potongan kode di Tugas memberikan lebih banyak kendali kepada pengguna untuk menyesuaikan model dan pelatihan. Potongan kode di Praktikum 2 lebih mudah digunakan karena menyediakan fitur-fitur terintegrasi yang siap pakai.
- Potongan kode di Tugas dapat membantu pengguna memahami cara kerja setiap langkah dalam pelatihan. Potongan kode di Praktikum 2 lebih fokus pada pemakaian yang lebih mudah dan efisien.